In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os
import pydicom
import cv2
from keras.preprocessing.image import ImageDataGenerator
import shutil

In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from numpy import load
from numpy import vstack
from matplotlib import pyplot
from numpy.random import randint
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from numpy import expand_dims
from skimage import morphology, io, color, exposure, img_as_float, transform


In [ ]:
def convert_to_array(file):
    ds = pydicom.dcmread(file)

    shape = ds.pixel_array.shape

    # Convert to float to avoid overflow or underflow losses.
    image_2d = ds.pixel_array.astype(float)

    # Rescaling grey scale between 0-255
    image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0

    # Convert to uint
    image_2d_scaled = np.uint8(image_2d_scaled)
    return image_2d_scaled

In [ ]:
def preprocess_for_segmentation(img,im_shape):
    img = transform.resize(img, im_shape)
    img = exposure.equalize_hist(img)
    img = np.expand_dims(img, -1)
    X = np.array(img)
    y = np.array(img)
    X -= X.mean()
    X /= X.std()
    return X

In [ ]:
def removal_of_white_text(img):
    ret,image=cv2.threshold(img,250,255,cv2.THRESH_TOZERO_INV)
    return image

In [ ]:
def crop_img(dcm_img_path, seg_model, InputShapeOfModel):
    # InputShapeOfModel is the input shape of the segmentation model
    xray_image = convert_to_array(dcm_img_path)
    xray_image = cv2.resize(xray_image, (1024, 1024))
    width_scale = xray_image.shape[1] // InputShapeOfModel[1] # width-scaling factor for resize
    height_scale = xray_image.shape[0] // InputShapeOfModel[0]  # height-scaling factor for resize
    # The scaling factors are later used to find the cropping region for original image size

    #segment the lung by seg_model
    img = preprocess_for_segmentation(xray_image, InputShapeOfModel)  # preprocessing the Xray image for segmentation
    inp_shape = img.shape
    X = np.expand_dims(img, axis=0)
    pred = seg_model.predict(X)[..., 0].reshape(inp_shape[:2])  # predicted segmentation
    
    # find bounding box around the two lungs
    ret, pr = cv2.threshold(pred, 0.95, 1, cv2.THRESH_BINARY)
    kernel = np.ones((3, 3), np.uint8)
    pr = np.array(pr * 255, dtype=np.uint8)
    pr = cv2.morphologyEx(pr, cv2.MORPH_OPEN, kernel, iterations=3)
    pr_canny = cv2.Canny(pr, 170, 255)
    cnts = cv2.findContours(pr_canny, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cntsSorted = sorted(cnts[0], key=lambda x: cv2.contourArea(x), reverse=True)
    x_c = []
    y_c = []
    b = 0
    for i in range(len(cntsSorted)):
        x, y, w, h = cv2.boundingRect(cntsSorted[i])
        if (w*h > 2500):
            x_c.append(x)
            x_c.append(x + w)
            y_c.append(y)
            y_c.append(y + h)
    crp_p1 = (max([min(x_c) - 20, 0]), max([min(y_c) - 20, 0]))
    crp_p2 = (min([max(x_c) + 20, pr.shape[0]]), min([max(y_c) + 20, pr.shape[1]]))
    # the crop area is scaled to the original image size
    x1 = int(crp_p1[0] * width_scale)
    y1 = int(crp_p1[1] * height_scale)
    x2 = int(crp_p2[0] * width_scale)
    y2 = int(crp_p2[1] * height_scale)
    h=y2-y1
    w=x2-x1
    if h > w:
        for a in range(x1+1):
            if (x2 - a + h - w) < xray_image.shape[1]:
#                 return x1-a, y1, x1+h, y2
                return xray_image[x1-a:x1+h, y1:y2]
    if w > h:
        for a in range(y1+1):
            if (y2 - a + w - h) < xray_image.shape[0]:
#                 return x1, y1-a, x2, y1+w
                return xray_image[x1:x2, y1-a:y1+w]
               
#     print(min(x_c))
#     print((min(x_c), min(y_c)), (max(x_c), max(y_c)))
#     print(crp_p1, crp_p2)
#     print((x1, y1), (x2, y2))
#     cv2.rectangle(pr, (min(x_c), min(y_c)), (max(x_c), max(y_c)), (0, 255, 0), 10)
#     cv2.rectangle(pr, crp_p1, crp_p2, (255, 0, 0), 5)
#     cv2.rectangle(xray_image, (x1, y1), (x2, y2), (255, 0, 0), 5)
#     plt.imshow(xray_image)
    return xray_image[x1:x2, y1:y2]

In [ ]:
seg_model = load_model('../input/cgan-model/CGAN_model.h5')
InputShapeOfModel = (512, 512)
img_path = '../input/rsna-pneumonia-detection-challenge/stage_2_train_images/0092d9c5-26b6-4e66-b196-49b2224ab8d1.dcm'
img = crop_img(img_path, seg_model, InputShapeOfModel)
plt.imshow(img)


In [ ]:
df = pd.read_csv("../input/rsna-pneumonia-detection-challenge/stage_2_detailed_class_info.csv")
normal_df = df[df['class'] == 'Normal']
abnormal_df = df[df['class'] != 'Normal']
_,normal_df_rand = train_test_split(normal_df, test_size = 6000/8851, shuffle=True)
_,abnormal_df_rand = train_test_split(abnormal_df, test_size = 6000/21376, shuffle=True)
my_df = shuffle(normal_df_rand.append(abnormal_df_rand))

In [ ]:
my_df

In [ ]:
train_df, test_df = train_test_split(my_df, test_size=0.3)
test_df, valid_df = train_test_split(test_df, test_size=0.5)
normal_train_df = train_df[train_df['class'] == 'Normal']
abnormal_train_df = train_df[train_df['class'] != 'Normal']
normal_valid_df = valid_df[valid_df['class'] == 'Normal']
abnormal_valid_df = valid_df[valid_df['class'] != 'Normal']
normal_test_df = test_df[test_df['class'] == 'Normal']
abnormal_test_df = test_df[test_df['class'] != 'Normal']


print('train_valid_test: %d, %d, %d' %(len(train_df), len(valid_df), len(test_df)))
print('train_normal/anormal: %d/%d' %(len(normal_train_df), len(abnormal_train_df)))
print('valid_normal/anormal: %d/%d' %(len(normal_valid_df), len(abnormal_valid_df)))
print('test_normal/anormal: %d/%d' %(len(normal_test_df), len(abnormal_test_df)))

In [ ]:
normal_train_list = list(normal_train_df['patientId'])
normal_valid_list = list(normal_valid_df['patientId'])
normal_test_list = list(normal_test_df['patientId'])

abnormal_train_list = list(abnormal_train_df['patientId'])
abnormal_valid_list = list(abnormal_valid_df['patientId'])
abnormal_test_list = list(abnormal_test_df['patientId'])

In [ ]:
# shutil.rmtree('RSNA_dataset')

In [ ]:
os.mkdir('RSNA_dataset')

train_path = os.path.join('./RSNA_dataset', 'train')
os.mkdir(train_path)
normal_train_path = os.path.join(train_path, 'normal')
os.mkdir(normal_train_path)
abnormal_train_path = os.path.join(train_path, 'abnormal')
os.mkdir(abnormal_train_path)

valid_path = os.path.join('./RSNA_dataset', 'valid')
os.mkdir(valid_path)
normal_valid_path = os.path.join(valid_path, 'normal')
os.mkdir(normal_valid_path)
abnormal_valid_path = os.path.join(valid_path, 'abnormal')
os.mkdir(abnormal_valid_path)

test_path = os.path.join('./RSNA_dataset', 'test')
os.mkdir(test_path)
normal_test_path = os.path.join(test_path, 'normal')
os.mkdir(normal_test_path)
abnormal_test_path = os.path.join(test_path, 'abnormal')
os.mkdir(abnormal_test_path)

In [ ]:
data_dir1 = '../input/rsna-pneumonia-detection-challenge/stage_2_test_images/'
data_dir2 = '../input/rsna-pneumonia-detection-challenge/stage_2_train_images/'

In [ ]:
for img in normal_train_list:
    if ((img + '.dcm') in os.listdir(data_dir1)):
        im_array = crop_img(data_dir1 + img + '.dcm', seg_model, InputShapeOfModel)
        dst = normal_train_path + '/' + img + '.png'
        cv2.imwrite(dst, im_array)
    if ((img + '.dcm') in os.listdir(data_dir2)):
        im_array = crop_img(data_dir2 + img + '.dcm', seg_model, InputShapeOfModel)
        dst = normal_train_path + '/' + img + '.png'
        cv2.imwrite(dst, im_array)
        
for img in abnormal_train_list:
    if ((img + '.dcm') in os.listdir(data_dir1)):
        im_array = crop_img(data_dir1 + img + '.dcm', seg_model, InputShapeOfModel)
        dst = abnormal_train_path + '/' + img + '.png'
        cv2.imwrite(dst, im_array)
    if ((img + '.dcm') in os.listdir(data_dir2)):
        im_array = crop_img(data_dir2 + img + '.dcm', seg_model, InputShapeOfModel)
        dst = abnormal_train_path + '/' + img + '.png'
        cv2.imwrite(dst, im_array)

In [ ]:
for img in normal_valid_list:
    if ((img + '.dcm') in os.listdir(data_dir1)):
        im_array = crop_img(data_dir1 + img + '.dcm', seg_model, InputShapeOfModel)
        dst = normal_valid_path + '/' + img + '.png'
        cv2.imwrite(dst, im_array)
    if ((img + '.dcm') in os.listdir(data_dir2)):
        im_array = crop_img(data_dir2 + img + '.dcm', seg_model, InputShapeOfModel)
        dst = normal_valid_path + '/' + img + '.png'
        cv2.imwrite(dst, im_array)
        
for img in abnormal_valid_list:
    if ((img + '.dcm') in os.listdir(data_dir1)):
        im_array = crop_img(data_dir1 + img + '.dcm', seg_model, InputShapeOfModel)
        dst = abnormal_valid_path + '/' + img + '.png'
        cv2.imwrite(dst, im_array)
    if ((img + '.dcm') in os.listdir(data_dir2)):
        im_array = crop_img(data_dir2 + img + '.dcm', seg_model, InputShapeOfModel)
        dst = abnormal_valid_path + '/' + img + '.png'
        cv2.imwrite(dst, im_array)

In [ ]:
for img in normal_test_list:
    if ((img + '.dcm') in os.listdir(data_dir1)):
        im_array = crop_img(data_dir1 + img + '.dcm', seg_model, InputShapeOfModel)
        dst = normal_test_path + '/' + img + '.png'
        cv2.imwrite(dst, im_array)
    if ((img + '.dcm') in os.listdir(data_dir2)):
        im_array = crop_img(data_dir2 + img + '.dcm', seg_model, InputShapeOfModel)
        dst = normal_test_path + '/' + img + '.png'
        cv2.imwrite(dst, im_array)
        
for img in abnormal_test_list:
    if ((img + '.dcm') in os.listdir(data_dir1)):
        im_array = crop_img(data_dir1 + img + '.dcm', seg_model, InputShapeOfModel)
        dst = abnormal_test_path + '/' + img + '.png'
        cv2.imwrite(dst, im_array)
    if ((img + '.dcm') in os.listdir(data_dir2)):
        im_array = crop_img(data_dir2 + img + '.dcm', seg_model, InputShapeOfModel)
        dst = abnormal_test_path + '/' + img + '.png'
        cv2.imwrite(dst, im_array)

In [ ]:
train_path = './RSNA_dataset/train/'

In [ ]:
NUM_AUG_IMAGES_WANTED = 8000

IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256

In [ ]:
class_list = ['normal','abnormal']
data_path = './RSNA_dataset/'
for item in class_list:
    aug_dir = os.path.join(data_path, 'aug_dir')
    os.mkdir(aug_dir)
    img_dir = os.path.join(aug_dir, 'img_dir')
    os.mkdir(img_dir)
    img_class = item
    img_list = os.listdir(train_path + img_class)
    for fname in img_list:
            src = os.path.join(train_path + img_class, fname)
            dst = os.path.join(img_dir, fname)
            shutil.copyfile(src, dst)
    path = aug_dir
    save_path = train_path + img_class

    datagen = ImageDataGenerator(
        rescale=1.0/255,
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 6,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False
        )  # randomly flip images

    batch_size = 50

    aug_datagen = datagen.flow_from_directory(path,save_to_dir=save_path,save_format='png',target_size=(IMAGE_HEIGHT,IMAGE_WIDTH),batch_size=batch_size)
    num_files = len(os.listdir(img_dir))
    num_batches = int(np.ceil((NUM_AUG_IMAGES_WANTED-num_files)/batch_size))

    for i in range(0,num_batches):
        imgs, labels = next(aug_datagen)
    shutil.rmtree(aug_dir)

In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255)
train_gen = datagen.flow_from_directory(train_path,
                                        target_size=(256,256),
                                        batch_size=10,
                                        class_mode='categorical')                                      
val_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(256,256),
                                        batch_size=10,
                                        class_mode='categorical')
test_gen = datagen.flow_from_directory(test_path,
                                        target_size=(256,256),
                                        batch_size=1,
                                        class_mode='categorical',
                                        shuffle=False)

In [ ]:

# With data augmentation to prevent overfitting and handling the imbalance in dataset
 
# datagen = ImageDataGenerator(
#         rescale=1.0/255,
#         featurewise_center=False,  # set input mean to 0 over the dataset
#         samplewise_center=False,  # set each sample mean to 0
#         featurewise_std_normalization=False,  # divide inputs by std of the dataset
#         samplewise_std_normalization=False,  # divide each input by its std
#         zca_whitening=False,  # apply ZCA whitening
#         rotation_range = 6,  # randomly rotate images in the range (degrees, 0 to 180)
#         zoom_range = 0.2, # Randomly zoom image 
#         width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#         height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#         horizontal_flip = True,  # randomly flip images
#         vertical_flip=False
#         )  # randomly flip images
# train_gen = datagen.flow_from_dataframe(train_df, data_path,
#                                         x_col = 'patientId',
#                                         y_col = 'class',
#                                         target_size=(256,256),
#                                         batch_size=10,
#                                         classes= [0, 1],
#                                         class_mode='categorical')
# datagen = ImageDataGenerator(rescale=1.0/255)
# valid_gen = datagen.flow_from_dataframe(valid_df, data_path,
#                                         x_col = 'patientId',
#                                         y_col = 'class',
#                                         target_size=(256,256),
#                                         batch_size=10,
#                                         classes= [0, 1],
#                                         class_mode='categorical')
# test_gen = datagen.flow_from_dataframe(test_df, data_path,
#                                         x_col = 'patientId',
#                                         y_col = 'class',
#                                         target_size=(256,256),
#                                         batch_size=1,
#                                         classes= [0, 1],
#                                         class_mode='categorical')


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate, Dropout
import tensorflow.keras.backend as K
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications.densenet import DenseNet121

In [ ]:
plt.imshow(cv2.imread('./RSNA_dataset/train/normal/' + os.listdir('./RSNA_dataset/train/normal')[200]))

In [ ]:
len(os.listdir('./RSNA_dataset/train/abnormal'))

In [ ]:
np.amax(cv2.imread('./RSNA_dataset/train/normal/' + os.listdir('./RSNA_dataset/train/normal')[2000]))

In [ ]:
model_d=DenseNet121(weights='imagenet',include_top=False, 
input_shape=(256, 256, 3)) 

x=model_d.output

x= GlobalAveragePooling2D()(x)
x= BatchNormalization()(x)
x= Dropout(0.3)(x)
x= Dense(1024,activation='relu')(x) 
x= Dense(512,activation='relu')(x) 
x= BatchNormalization()(x)
x= Dropout(0.3)(x)

preds=Dense(2,activation='softmax')(x) 

model=Model(inputs=model_d.input,outputs=preds) 

In [ ]:
model.summary()

In [ ]:
model.compile(Adam(learning_rate=0.00001), loss='categorical_crossentropy', 
              metrics=['accuracy'])

# model.compile(SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD"), loss='categorical_crossentropy', 
#               metrics=['accuracy'])
 

In [ ]:
import tensorflow as tf
filepath = "./DenseNet121.h5"
keras_callbacks   = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min', min_delta=0.0001),
    tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
]
history = model.fit(train_gen, steps_per_epoch=200, validation_data=val_gen,epochs=200, verbose=1, callbacks = keras_callbacks)

In [ ]:
print('Plot accuracy: ')
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
print('Plot Loss')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# test


In [ ]:
predictions = model.predict_generator(test_gen, verbose = 1)

In [ ]:
predictions.argmax(axis = 1)

In [ ]:
test_gen.class_indices

In [ ]:
print('TEST: ')
scores = model.evaluate_generator(test_gen,1797)
print("Test Accuracy = ", scores[1])
print("Test Loss = ", scores[0])

In [ ]:
test_labels = test_gen.classes

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm = confusion_matrix(test_labels, predictions.argmax(axis = 1))
sns.heatmap(cm, annot = True)

In [ ]:
pip install openpyxl

In [ ]:
excel_data_df = pd.read_excel('../input/mycxr/image_test/export_df.xlsx', sheet_name='Sheet1')

In [ ]:
excel_data_df['path_image'] = excel_data_df['patient_id'].apply(str) + '-' + excel_data_df['accession_num'] + '.dcm'

In [ ]:
normal_image_list = list(excel_data_df['path_image'])

In [ ]:
inputdir = '../input/mycxr/image_test/test_image/'
outdir = './'
my_test_dir = os.path.join(outdir, 'my_test_image')
os.mkdir(my_test_dir)
normal_dir = os.path.join(my_test_dir, 'normal')
os.mkdir(normal_dir)
abnormal_dir = os.path.join(my_test_dir, 'abnormal')
os.mkdir(abnormal_dir)

In [ ]:
normal_dir

In [ ]:
abnormal_dir

In [ ]:
inputdir

In [ ]:
test_list = os.listdir(inputdir)

In [ ]:
for im in test_list:
    img = crop_img(inputdir + im, seg_model, InputShapeOfModel)
    if im in normal_image_list:
        cv2.imwrite(normal_dir + '/' + im.replace('.dcm','.png'),img)
    else:
        cv2.imwrite(abnormal_dir + '/' + im.replace('.dcm','.png'),img)

In [ ]:
plt.imshow(cv2.imread('./my_test_image/normal/18006863-21.0204.007245.png'))

In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255)
test_gen = datagen.flow_from_directory(my_test_dir,
                                        target_size=(256,256),
                                        batch_size=1,
                                        class_mode='categorical',
                                        shuffle=False)

In [ ]:
test_gen.class_indices

In [ ]:
test_labels = test_gen.classes
predictions = model.predict_generator(test_gen, verbose = 1)
predictions.argmax(axis = 1)
cm = confusion_matrix(test_labels, predictions.argmax(axis = 1))
test_gen.class_indices
sns.heatmap(cm, annot = True)

GRAD CAM

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
import cv2
# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pydicom

In [ ]:
mytest_normal_img = os.listdir('./my_test_image/normal/')
mytest_abnormal_img = os.listdir('./my_test_image/abnormal/')

In [ ]:
model_builder = keras.applications.xception.Xception
img_size = (256, 256)
preprocess_input = keras.applications.xception.preprocess_input
decode_predictions = keras.applications.xception.decode_predictions

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
def save_and_display_gradcam(img_path, heatmap, cam_path="gradcam.jpg", alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)
    
    return jet_heatmap, superimposed_img


In [ ]:
test_path = './my_test_image/abnormal/' + mytest_abnormal_img[80]

# Prepare image
img_array = preprocess_input(get_img_array(test_path, size=img_size))

# Remove last layer's softmax
#model.layers[-1].activation = None

# Print what the top predicted class is
preds = model.predict(img_array)
print('Prediction Result: ', list(test_gen.class_indices.keys())[np.where(preds[0] == np.amax(preds[0]))[0][0]])



# Generate class activation heatmap
last_conv_layer_name = "conv5_block16_concat"
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Display heatmap
print('GRAD CAM: ')
jet_heatmap, superimposed_img = save_and_display_gradcam(test_path, heatmap)
fig = plt.figure(figsize=(20, 20))
plt.subplot(2,2,1)
plt.imshow(heatmap)
plt.subplot(2,2,2)
plt.imshow(superimposed_img)

b,g,r = cv2.split(jet_heatmap)
gray = 0.299 * r + 0.587 * g + 0.114 * b
gray = np.asarray(r, np.uint8)
(T, thresh) = cv2.threshold(gray, 200, np.amax(gray), cv2.THRESH_BINARY)
pr_canny = cv2.Canny(thresh, 170, np.amax(thresh))
cnts = cv2.findContours(pr_canny, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cntsSorted = sorted(cnts[0], key=lambda x: cv2.contourArea(x), reverse=True)
# find bounding box
img = cv2.imread(test_path)
plt.subplot(2,2,4)
plt.imshow(img)
for i in range(len(cntsSorted)):
    x, y, w, h = cv2.boundingRect(cntsSorted[i])
    cv2.rectangle(img, (x, x+w), (y, y+h), (0, 255, 0), 5)
    

In [ ]:
test_path = './my_test_image/abnormal/' + mytest_abnormal_img[10]

# Prepare image
img_array = preprocess_input(get_img_array(test_path, size=img_size))

# Remove last layer's softmax
#model.layers[-1].activation = None

# Print what the top predicted class is
preds = model.predict(img_array)
print('Prediction Result: ', list(test_gen.class_indices.keys())[np.where(preds[0] == np.amax(preds[0]))[0][0]])



# Generate class activation heatmap
last_conv_layer_name = "conv5_block16_concat"
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Display heatmap
print('GRAD CAM: ')
jet_heatmap, superimposed_img = save_and_display_gradcam(test_path, heatmap)
fig = plt.figure(figsize=(20, 20))
plt.subplot(2,2,1)
plt.imshow(heatmap)
plt.subplot(2,2,2)
plt.imshow(superimposed_img)

b,g,r = cv2.split(jet_heatmap)
gray = 0.299 * r + 0.587 * g + 0.114 * b
gray = np.asarray(r, np.uint8)
(T, thresh) = cv2.threshold(gray, 200, np.amax(gray), cv2.THRESH_BINARY)
pr_canny = cv2.Canny(thresh, 170, np.amax(thresh))
cnts = cv2.findContours(pr_canny, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cntsSorted = sorted(cnts[0], key=lambda x: cv2.contourArea(x), reverse=True)
# find bounding box
img = cv2.imread(test_path)
plt.subplot(2,2,4)
plt.imshow(img)
for i in range(len(cntsSorted)):
    x, y, w, h = cv2.boundingRect(cntsSorted[i])
    cv2.rectangle(img, (x, x+w), (y, y+h), (0, 255, 0), 5)
    